In [1]:
# as_default主要在两个地方都有使用：
#           1、tf.Graph()与tf.Graph().as_default()
#           2、tf.Session()与tf.Session().as_default() 

#一般来说，图在一个网络里应该只会使用一个，故而用不用tf.Graph().as_default()都无所谓
#但是会话可能会在多处使用相同的会话，故而使用一下tf.Session().as_default()会比较方便

In [2]:
import tensorflow as tf

D:\Anaconda\Initial\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#先说第一点
#tensorflow当前线程都会在默认的graph上进行画图
#如果创建了另一个图，想要在另一个图上进行作画，那么就要将另一幅图变成当前的默认图才可以
#如果全局并没有创建图，因为默认会创建一张图，那么设置和不设置都无所谓

a=tf.constant([1.0,2.0],name="a")
b=tf.constant([3.0,4.0],name="b")
c=a+b
print(c.graph)
print(tf.get_default_graph())
assert c.graph == tf.get_default_graph()  #说明当前的图即是默认的图

print(tf.Graph())  #这里相当于又生出新的图，但是没有去使用
print(tf.Graph())  #同样的，执行一次，即相当于生出一个

In [4]:
print(tf.get_default_graph()) #先打印一下当前的默认图
g = tf.Graph()  #新生一个新图
with g.as_default():  #将该新图置为默认图
    c = tf.constant(5.0)  #在图中定义节点
    assert c.graph is g
    print(tf.get_default_graph()) #再打印一下当前的默认图

In [5]:
# 再说第二点
# session相当于会话，默认也是在当前图上的会话。
# tf.Session()表示一个临时会话，一旦出去了，哪怕会话没关，也会自动关闭
# tf.Session().as_default() 则表示将临时会话变成默认会话，默认会话默认情况下会随着图一直存在
#                           故而哪怕出去会话了，只要没有手动关闭，也可以继续使用

a = tf.constant(1.0)
b = tf.constant(2.0)
with tf.Session() as sess:
    print(a.eval())   
    
try:
    b.eval(session=sess) #此时已经出了临时会话，即表示临时会话已经关闭，再想使用会报错
except RuntimeError:
    print(RuntimeError)

print("="*20)
    
with tf.Session().as_default() as sess: #此时sess变成了默认会话，没有主动关闭，会一直存活
    print(a.eval())   
print(b.eval(session=sess)) #此时也能继续使用这个会话

print("="*20)

with tf.Session().as_default() as sess:
    print(a.eval())  
    sess.close() #此时主动关闭了默认会话，故而外面也无法直接继续使用了
try:
    b.eval(session=sess) #虽然是默认会话，但是已经被关闭了，故而也没法使用了
except RuntimeError:
    print(RuntimeError)

1.0
<class 'RuntimeError'>
1.0
2.0
1.0
<class 'RuntimeError'>
